In [1]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobu

In [2]:
from ortools.linear_solver import pywraplp
import math

In [3]:
# Create the ILP solver (SCIP)
# Details: https://developers.google.com/optimization/mip/mip_example?hl=ko
solver = pywraplp.Solver.CreateSolver("SAT")
if not solver:
    raise ImportError

In [4]:
# Create the decision variables.

num_of_orders = {"classic": 100, "bacon_cheese": 220, "chicken": 60}

sum_of_num_of_orders = sum(num_of_orders.values())

grills = ["A", "B"]
G = {}

for grill in grills:
  G[grill] = solver.IntVar(0.0, sum_of_num_of_orders, "G_"+grill)

x = {}
burgers_on_grill = ["x2_classic_A", "x2_classic_B", "x2_bacon_cheese_A", "x2_bacon_cheese_B", "x3_bacon_cheese_A", "x3_bacon_cheese_B", "x3_chicken_A", "x3_chicken_B"]

for burger_on_grill in burgers_on_grill:
  for type_of_burger in num_of_orders.keys():
    if type_of_burger in burger_on_grill:
      x[burger_on_grill] = solver.IntVar(0.0, num_of_orders[type_of_burger], burger_on_grill)

In [5]:
# Create the contraints
# Integer Linear Programming condition has been added from scratch.

# contraints are not made in the form of dictionaries, since it highly depends
# on the state of decision variables.

# The Number Of Orders
constraint_order_classic_2 = solver.Add(x["x2_classic_A"] + x["x2_classic_B"] == num_of_orders["classic"])
constraint_order_bacon_2 = solver.Add(x["x2_bacon_cheese_A"] + x["x2_bacon_cheese_B"] == num_of_orders["bacon_cheese"])
constraint_order_bacon_3 = solver.Add(x["x3_bacon_cheese_A"] + x["x3_bacon_cheese_B"] == num_of_orders["bacon_cheese"])
constraint_order_chicken_3 = solver.Add(x["x3_chicken_A"] + x["x3_chicken_B"] == num_of_orders["chicken"])

# The Availability of Grill B
constraint_unavail_grill_B = solver.Add(x["x3_bacon_cheese_B"] == 0)

# Time Constraints
constraint_time_grill_A = solver.Add(-720 * G["A"] + 20 * x["x2_classic_A"] + 15 * x["x2_bacon_cheese_A"] + 13 * x["x3_bacon_cheese_A"] + 17 * x["x3_chicken_A"] <= 0)
constraint_time_grill_B = solver.Add(-720 * G["B"] + 10 * x["x2_classic_B"] + 8 * x["x2_bacon_cheese_B"] + 0 * x["x3_bacon_cheese_B"] + 11 * x["x3_chicken_B"] <= 0)

In [6]:
# Set the objective function
unit_price_per_grill = {"A": 120, "B": 180}

solver.Minimize(solver.Sum(unit_price_per_grill[grill] * G[grill] for grill in grills))

In [9]:
# solve the model

print(f'Solving with {solver.SolverVersion()}')
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print("Solution:")
  print("Objective value =", solver.Objective().Value())
  for grill in grills:
    print(f"{grill} = {G[grill].solution_value()}")
  for burger_on_grill in burgers_on_grill:
    print(f"{burger_on_grill} = {x[burger_on_grill].solution_value()}")
else:
    print("The problem does not have an optimal solution.")

Solving with CP-SAT solver v9.10.4067
Solution:
Objective value = 1380.0
A = 7.0
B = 3.0
x2_classic_A = 0.0
x2_classic_B = 100.0
x2_bacon_cheese_A = 77.0
x2_bacon_cheese_B = 143.0
x3_bacon_cheese_A = 220.0
x3_bacon_cheese_B = 0.0
x3_chicken_A = 60.0
x3_chicken_B = 0.0
